In [8]:
import os.path

import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

In [9]:
import sys
sys.path.append('./drive/MyDrive/Projects/MiniSoundFinder/lib/')

import classifiers

In [10]:
experiment_path = './drive/MyDrive/Projects/MiniSoundFinder/experiments/basic_convnet_01'

model = classifiers.BasicConvNet(input_channels=1, output_size=10)
weights_path = os.path.join(experiment_path, 'model.pth')
model.load_state_dict(torch.load(weights_path, map_location=torch.device("cpu")))
model.eval()

BasicConvNet(
  (features): Sequential(
    (0): Conv1d(1, 4, kernel_size=(80,), stride=(64,), bias=False)
    (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(4, 16, kernel_size=(3,), stride=(4,), bias=False)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(16, 16, kernel_size=(3,), stride=(4,), bias=False)
    (7): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): AdaptiveMaxPool1d(output_size=16)
  )
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)

In [12]:
MAX_LENGTH = 384000

sample_input = torch.distributions.uniform.Uniform(-10000, 10000).sample((1, 1, MAX_LENGTH))

In [13]:
traced_script_module = torch.jit.trace(model, sample_input)

In [14]:
traced_script_module_optimized = optimize_for_mobile(traced_script_module)

In [15]:
output_path = './drive/MyDrive/Projects/MiniSoundFinder/exports/'


traced_script_module_optimized._save_for_lite_interpreter(
    os.path.join(output_path, 'basic_convnet.pt'))